[Rendered Notebook](http://nbviewer.jupyter.org/github/andersonfrailey/Notebook-Uploads/blob/master/newcpswts.ipynb)

In [20]:
import pandas as pd
import numpy as np
import taxcalc as tc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from bokeh.layouts import layout, row, gridplot, column
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [21]:
years = [i for i in range(2014, 2028)]

In [22]:
cps = pd.read_csv('cps_data/cps.csv.gz', compression='gzip')
new_wts = pd.read_csv('cps_stage2/cps_weights.csv.gz', compression='gzip')

In [23]:
new_wts.head()

,WT2014,WT2015,WT2016,WT2017,WT2018,WT2019,WT2020,WT2021,WT2022,WT2023,WT2024,WT2025,WT2026,WT2027
0,23002,23519,23890,24301,24681,25014,25309,25608,25895,26185,26477,26774,27075,27377
1,19460,19898,20211,20559,20881,21163,21412,21665,21908,22153,22400,22651,22906,23161
2,24528,28361,29303,30272,31287,32350,33466,34564,35707,36838,37939,39085,40169,41182
3,17133,26277,17794,18101,18383,18632,18851,19074,19288,19504,19721,19942,20167,20391
4,26012,30078,31077,32104,33181,34308,35491,36657,37869,39068,40236,41451,42600,43675


In [24]:
base = tc.Calculator(records=tc.Records.cps_constructor(),
                     policy=tc.Policy())
new = tc.Calculator(records=tc.Records(data=cps, weights=new_wts, start_year=2014),
                    policy=tc.Policy())

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [25]:
# collect data for each of the years
base_data = {}
new_data = {}
var_list = ['s006', 'iitax', 'combined', 'payrolltax', 'c00100',
            'expanded_income']
for year in years:
    base.advance_to_year(year)
    base.calc_all()
    new.advance_to_year(year)
    new.calc_all()
    base_data[year] = base.dataframe(var_list)
    new_data[year] = new.dataframe(var_list)

In [26]:
# tax liability by year in 2014
data = {'year': [], 'combined_base': [], 'combined_new': [], 'combined_change': [],
        'combined_pct_change': [], 'iitax_pct_change': [], 'payroll_pct_change': [],
        'iitax_base': [], 'iitax_new': [], 'iitax_change': [],
        'payroll_base': [], 'payroll_new': [], 'payroll_change': []}
for year in years:
    data['year'].append(year)
    combined_new = (new_data[year]['combined'] * new_data[year]['s006']).sum()
    combined_base = (base_data[year]['combined'] * base_data[year]['s006']).sum()
    combined_change = combined_new - combined_base
    combined_pct_change = (combined_change / combined_base) * 100
    
    iitax_new = (new_data[year]['iitax'] * new_data[year]['s006']).sum()
    iitax_base = (base_data[year]['iitax'] * base_data[year]['s006']).sum()
    iitax_change = iitax_new - iitax_base
    iitax_pct_change = (iitax_change / iitax_base) * 100

    payroll_new = (new_data[year]['payrolltax'] * new_data[year]['s006']).sum()
    payroll_base = (base_data[year]['payrolltax'] * base_data[year]['s006']).sum()
    payroll_change = payroll_new - payroll_base
    payroll_pct_change = (payroll_change / payroll_base) * 100

    data['combined_base'].append(combined_base)
    data['combined_new'].append(combined_new)
    data['combined_change'].append(combined_change)
    data['combined_pct_change'].append(combined_pct_change)
    data['iitax_base'].append(iitax_base)
    data['iitax_new'].append(iitax_new)
    data['iitax_change'].append(iitax_change)
    data['iitax_pct_change'].append(iitax_pct_change)
    data['payroll_base'].append(payroll_base)
    data['payroll_new'].append(payroll_new)
    data['payroll_change'].append(payroll_change)
    data['payroll_pct_change'].append(payroll_pct_change)
cds = ColumnDataSource(data)
data_df = pd.DataFrame(data, index=data['year'])

In [27]:
f = figure(title='Tax Liability by Year')
f.line(x='year', y='combined_base', line_width=2, color='blue', source=cds, legend='Combined-Base')
f.line(x='year', y='combined_new', line_width=2, color='lightblue', source=cds, legend='Combined-new')
f.line(x='year', y='iitax_base', line_width=2, color='green', source=cds, legend='IITax-Base')
f.line(x='year', y='iitax_new', line_width=2, color='lime', source=cds, legend='IITax-New')
f.line(x='year', y='payroll_base', line_width=2, color='red', source=cds, legend='Payroll-Base')
f.line(x='year', y='payroll_new', line_width=2, color='pink', source=cds, legend='Payroll-New')
f.legend.location='top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
show(f)

In [28]:
pd.options.display.float_format = '{:,.2f}'.format
print('Combined Tax Liability')
data_df[['combined_base', 'combined_new', 'combined_change', 'combined_pct_change']].transpose()

Combined Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
combined_base,"2,049,761,794,852.92","2,110,841,250,755.81","2,192,101,336,640.05","2,328,589,406,483.97","2,326,852,347,902.63","2,455,208,000,725.77","2,578,843,260,672.17","2,711,805,209,706.87","2,853,027,976,383.55","3,001,258,288,406.50","3,155,871,943,780.05","3,317,717,564,356.19","3,682,346,347,868.92","3,862,491,374,617.44"
combined_new,"2,084,831,791,776.91","2,295,110,576,180.62","2,370,532,771,117.32","2,503,099,387,988.52","2,489,320,156,856.33","2,614,023,314,335.22","2,733,363,461,926.12","2,861,052,957,906.73","2,996,785,636,297.09","3,138,967,785,113.46","3,284,019,034,842.64","3,427,805,535,299.55","3,785,096,780,502.17","3,948,583,315,675.52"
combined_change,"35,069,996,924.00","184,269,325,424.81","178,431,434,477.27","174,509,981,504.55","162,467,808,953.70","158,815,313,609.45","154,520,201,253.95","149,247,748,199.86","143,757,659,913.55","137,709,496,706.96","128,147,091,062.58","110,087,970,943.36","102,750,432,633.25","86,091,941,058.08"
combined_pct_change,1.71,8.73,8.14,7.49,6.98,6.47,5.99,5.50,5.04,4.59,4.06,3.32,2.79,2.23


In [29]:
print('Income Tax Liability')
data_df[['iitax_base', 'iitax_new', 'iitax_change', 'iitax_pct_change']].transpose()

Income Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
iitax_base,"1,052,768,062,368.58","1,079,321,792,276.94","1,118,073,598,335.02","1,194,682,943,874.67","1,134,225,986,055.48","1,207,623,350,219.63","1,278,600,912,769.22","1,355,456,344,576.62","1,435,537,108,564.83","1,518,783,771,055.15","1,605,135,131,486.43","1,695,298,234,281.98","1,984,937,041,617.15","2,086,678,633,847.68"
iitax_new,"1,085,451,650,467.22","1,225,642,282,849.22","1,262,593,055,473.18","1,337,381,627,474.41","1,268,817,324,182.53","1,343,705,638,699.74","1,416,819,497,983.29","1,495,527,798,011.77","1,578,119,381,298.53","1,664,048,618,178.26","1,750,403,286,831.14","1,833,195,689,987.18","2,127,218,857,253.27","2,225,058,758,581.44"
iitax_change,"32,683,588,098.65","146,320,490,572.28","144,519,457,138.17","142,698,683,599.74","134,591,338,127.05","136,082,288,480.11","138,218,585,214.08","140,071,453,435.15","142,582,272,733.70","145,264,847,123.11","145,268,155,344.72","137,897,455,705.20","142,281,815,636.12","138,380,124,733.76"
iitax_pct_change,3.10,13.56,12.93,11.94,11.87,11.27,10.81,10.33,9.93,9.56,9.05,8.13,7.17,6.63


In [30]:
print('Payroll Tax Liability')
data_df[['payroll_base', 'payroll_new', 'payroll_change', 'payroll_pct_change']].transpose()

Payroll Tax Liability


,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027
payroll_base,"996,993,732,484.34","1,031,519,458,478.87","1,074,027,738,305.03","1,133,906,462,609.30","1,192,626,361,847.16","1,247,584,650,506.14","1,300,242,347,902.95","1,356,348,865,130.26","1,417,490,867,818.72","1,482,474,517,351.36","1,550,736,812,293.63","1,622,419,330,074.21","1,697,409,306,251.77","1,775,812,740,769.76"
payroll_new,"999,380,141,309.69","1,069,468,293,331.40","1,107,939,715,644.13","1,165,717,760,514.11","1,220,502,832,673.80","1,270,317,675,635.47","1,316,543,963,942.83","1,365,525,159,894.96","1,418,666,254,998.56","1,474,919,166,935.20","1,533,615,748,011.49","1,594,609,845,312.38","1,657,877,923,248.91","1,723,524,557,094.08"
payroll_change,"2,386,408,825.35","37,948,834,852.52","33,911,977,339.10","31,811,297,904.82","27,876,470,826.64","22,733,025,129.33","16,301,616,039.88","9,176,294,764.70","1,175,387,179.85","-7,555,350,416.16","-17,121,064,282.13","-27,809,484,761.83","-39,531,383,002.87","-52,288,183,675.68"
payroll_pct_change,0.24,3.68,3.16,2.81,2.34,1.82,1.25,0.68,0.08,-0.51,-1.10,-1.71,-2.33,-2.94


In [31]:
def hist_plots(base_data, new_data, year):
    change_wts = new_data['s006'] - base_data['s006']
    pct_change_wts = (change_wts / base_data['s006']) * 100
    hist_diff, edges_diff = np.histogram(change_wts, density=True, bins=50)
    hist_pct, edges_pct = np.histogram(pct_change_wts, density=True, bins=50)
    
    # create plots
    f1 = figure(title=f'Change in Weights for {year}', width=350, height=350)  # difference
    f1.quad(top=hist_diff, bottom=0, left=edges_diff[:-1], right=edges_diff[1:],
            fill_color='cyan', line_color='black')
    f1.yaxis.axis_label = 'Frequency'
    f1.xaxis.axis_label = 'Change'

    f2 = figure(title=f'Pct Change in Weights for {year}', width=350, height=350)  # pct change
    f2.quad(top=hist_pct, bottom=0, left=edges_pct[:-1], right=edges_pct[1:],
            fill_color='cyan', line_color='black')
    f2.yaxis.axis_label = 'Frequency'
    f2.xaxis.axis_label = 'Pct Change'
    
    return f1, f2

In [32]:
# analyze change in each record's weight
plot_list_diff = []  # holds graphs showing difference in wt
plot_list_pct = []  # holds graphs showing histogram of pct change in wt
for year in years:
    base_data1 = base_data[year]
    new_data1 = new_data[year]

    hist_diff1, hist_pct1 = hist_plots(base_data1, new_data1, year)
    plot_list_diff.append(hist_diff1)
    plot_list_pct.append(hist_pct1)

In [33]:
show(column(plot_list_diff))

In [34]:
show(column(plot_list_pct))